# Notebook for generating the Production Demo for Categorical Age Prediction of Domestic Felines (kitten, adult, senior)

Three cat_ids are selected that each have 7 contributions. 

Demo samples are removed from training set and model is built on remaining data. 

Demo samples are available for the production demo in https://github.com/aster-droide/age-prediction-demo-categorical

In [180]:
# Standard imports
import numpy as np
import pandas as pd
import random
from datetime import datetime
from collections import Counter

# Sklearn imports
from sklearn.model_selection import train_test_split, GroupShuffleSplit, GroupKFold, StratifiedGroupKFold
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, RobustScaler, StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from sklearn.utils.class_weight import compute_class_weight
from sklearn.inspection import permutation_importance

# Imbalanced-learn import
from imblearn.over_sampling import SMOTE

# TensorFlow and Keras imports
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Input, BatchNormalization, concatenate
from tensorflow.keras.optimizers import Adam, RMSprop, SGD, Adamax, AdamW
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from keras.regularizers import l1, l2, L1L2

# Optuna import
import optuna

# Visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns

# to save the scaler
import joblib

In [181]:
# Set a fixed random seed for reproducibility
random.seed(42) 
np.random.seed(42)
tf.random.set_seed(42)

# Load datasets
dataframe = pd.read_csv('/Users/astrid/PycharmProjects/audioset-thesis-work/audioset/vggish/embeddings/8april_looped_embeddings.csv')

dataframe.drop('mean_freq', axis=1, inplace=True)

def assign_age_group(age, age_groups):
    for group_name, age_range in age_groups.items():
        if age_range[0] <= age < age_range[1]:
            return group_name
    return 'Unknown'  # For any age that doesn't fit the defined groups

# Define age groups
age_groups = {
    'kitten': (0, 0.5),
    'adult': (0.5, 10),
    'senior': (10, 20)
}

# Create a new column for the age group
dataframe['age_group'] = dataframe['target'].apply(assign_age_group, age_groups=age_groups)

print(dataframe['age_group'].value_counts())

adult     460
senior    306
kitten    171
Name: age_group, dtype: int64


# save demo rows to external csv

In [182]:
# Select all rows corresponding to the specified cat_id values
selected_cat_ids = ['117A', '099A', '050A']
demo_samples = dataframe[dataframe['cat_id'].isin(selected_cat_ids)]

In [183]:
demo_samples

,0,1,2,3,4,5,6,7,8,9,...,122,123,124,125,126,127,gender,target,cat_id,age_group
115,2712.3545,-1135.1974,-2841.0250,107.47739,140.551970,4600.0815,-1595.8768,-1847.3910,5690.7710,1924.2240,...,2385.6099,-466.91070,-2724.2540,3201.6511,-5709.4130,-998.5291,F,5.25,099A,adult
202,3426.3960,-1453.4320,-3635.5828,178.96103,306.443570,5830.9550,-2044.4401,-2213.7430,7086.1597,2355.9998,...,3041.8423,-527.45470,-3442.1597,4118.9020,-7263.3790,-1356.3002,F,18.00,117A,senior
209,3355.9521,-1303.4120,-3631.6050,141.45258,244.160480,5786.3228,-2031.5255,-2233.3796,7078.1445,2368.2544,...,3040.0510,-640.22815,-3344.3310,3989.8276,-7187.9310,-1249.6372,X,0.00,050A,kitten
210,2545.7540,-1022.4693,-2688.3418,83.26001,181.085170,4263.4214,-1531.0897,-1716.8507,5238.8730,1717.6139,...,2250.8567,-462.13022,-2463.6191,2999.9104,-5323.4185,-910.9742,X,0.00,050A,kitten
211,2886.9104,-1155.1565,-3081.1362,104.03778,181.506550,4943.7730,-1759.0626,-1962.9388,6099.0923,1992.6163,...,2565.9219,-504.01306,-2830.1445,3407.9817,-6132.1660,-1061.8490,X,0.00,050A,kitten
215,3212.6990,-1365.4951,-3333.8115,174.96430,375.178700,5477.5283,-1932.7018,-2047.5621,6611.2944,2187.4224,...,2755.9788,-451.63200,-3125.1729,3826.0570,-6694.3270,-1277.1831,F,18.00,117A,senior
239,3224.6812,-1306.4841,-3483.2856,155.92056,214.709520,5574.1610,-1957.7963,-2125.3423,6849.4150,2297.2240,...,2919.4226,-584.55370,-3240.3708,3825.4539,-6930.2627,-1185.7925,X,0.00,050A,kitten
275,2827.5178,-1220.6775,-2959.4004,150.27759,115.613266,4793.8667,-1631.2704,-1904.5444,5927.4536,1959.3539,...,2474.9430,-398.57608,-2907.0308,3327.1280,-5999.8540,-1065.6439,F,18.00,117A,senior
287,3100.4272,-1366.3661,-3288.3145,108.66681,185.973920,5313.1400,-1850.2406,-2113.6880,6546.9883,2256.5942,...,2731.5945,-551.58750,-3216.2427,3692.1833,-6532.3560,-1190.3914,F,5.25,099A,adult
292,3376.4229,-1375.8116,-3668.7173,135.95068,188.348180,5808.9460,-2060.1282,-2271.3206,7147.0750,2407.6265,...,3057.3933,-641.95850,-3405.4680,4017.4436,-7231.2450,-1256.0560,X,0.00,050A,kitten


In [184]:
# Initialize and fit the label encoder
label_encoder = LabelEncoder()
dataframe['label'] = label_encoder.fit_transform(dataframe['age_group'].values)

In [185]:
dataframe.head(5)

,0,1,2,3,4,5,6,7,8,9,...,123,124,125,126,127,gender,target,cat_id,age_group,label
0,3253.6790,-1300.0604,-3428.6190,178.22336,145.87761,5530.4010,-1929.7223,-2155.6733,6838.9844,2277.8290,...,-530.91520,-3267.7144,3789.5164,-6954.0560,-1200.4550,M,2.0,006A,adult,0
1,3338.0847,-1419.9960,-3464.4106,183.58270,311.41168,5724.5674,-1989.8912,-2187.1287,7025.0780,2406.5380,...,-515.08560,-3226.8980,3920.7097,-7107.8330,-1304.0648,F,5.0,000A,adult,0
2,3282.3360,-1396.4010,-3533.9820,149.29416,207.89177,5654.8940,-1989.5737,-2193.4783,6968.3830,2366.7522,...,-593.87024,-3310.9148,3889.7998,-7059.0030,-1274.8529,X,0.0,044A,kitten,1
3,4882.2915,-2161.8300,-5307.8610,168.99500,255.57112,8415.0170,-2979.1380,-3213.3972,10388.6070,3472.3523,...,-883.19380,-4949.7915,5769.6240,-10496.9030,-2006.7511,X,0.0,014B,kitten,1
4,3503.6260,-1458.7937,-3623.8113,196.71686,237.97202,5886.2270,-2068.3577,-2297.6812,7219.8496,2454.4438,...,-546.66240,-3363.1080,4081.6120,-7353.6616,-1369.3765,F,5.0,000A,adult,0


## save embeddings and labels from demo set to .txt

In [186]:
# Ensure the target labels in demo_samples are encoded using the same LabelEncoder
demo_samples = demo_samples.copy()  
demo_samples['label'] = label_encoder.transform(demo_samples['age_group'].values)

# Extract features and labels from demo_samples
features = demo_samples.iloc[:, :-5].values 
labels = demo_samples['label'].values

# Save each row to a separate CSV file
for i, (feature_row, label) in enumerate(zip(features, labels)):
    # Create a DataFrame for the current row
    row_df = pd.DataFrame([np.append(feature_row, label)])
    
    # Create a filename
    filename = f'demo_sample_{i}.csv'
    
    # Save to CSV file
    row_df.to_csv(filename, index=False, header=False)
    
    print(f'Saved {filename}')

Saved demo_sample_0.csv
Saved demo_sample_1.csv
Saved demo_sample_2.csv
Saved demo_sample_3.csv
Saved demo_sample_4.csv
Saved demo_sample_5.csv
Saved demo_sample_6.csv
Saved demo_sample_7.csv
Saved demo_sample_8.csv
Saved demo_sample_9.csv
Saved demo_sample_10.csv
Saved demo_sample_11.csv
Saved demo_sample_12.csv
Saved demo_sample_13.csv
Saved demo_sample_14.csv
Saved demo_sample_15.csv
Saved demo_sample_16.csv
Saved demo_sample_17.csv
Saved demo_sample_18.csv
Saved demo_sample_19.csv
Saved demo_sample_20.csv


In [187]:
# Ensure the target labels are encoded as 0 for kitten and 1 for senior
demo_samples = demo_samples.copy()  # Avoid SettingWithCopyWarning
demo_samples['label'] = label_encoder.transform(demo_samples['age_group'].values)

# Extract features and labels
features = demo_samples.iloc[:, :-5].values
labels = demo_samples['label'].values

# Combine features and labels into a single DataFrame
combined_data = np.hstack((features, labels.reshape(-1, 1)))
combined_df = pd.DataFrame(combined_data)

# Create a filename for the combined CSV file
combined_filename = 'combined_demo_samples.csv'

# Save the combined data to a single CSV file
combined_df.to_csv(combined_filename, index=False, header=False)

print(f'Saved {combined_filename}')

Saved combined_demo_samples.csv


In [188]:
# Count the occurrences of each cat_id
cat_id_counts = dataframe['cat_id'].value_counts().reset_index()
cat_id_counts.columns = ['cat_id', 'count']

# Merge with the age group information
age_group_info = dataframe[['cat_id', 'age_group']].drop_duplicates()
cat_id_counts_with_age_group = cat_id_counts.merge(age_group_info, on='cat_id')

pd.set_option('display.max_rows', None)

# Display the result
cat_id_counts_with_age_group.sort_values(by='count', ascending=True)

,cat_id,count,age_group
111,026B,1,adult
92,019B,1,adult
93,110A,1,kitten
94,100A,1,adult
95,090A,1,senior
96,115A,1,kitten
97,091A,1,senior
98,024A,1,senior
99,073A,1,adult
100,066A,1,adult


### samples for demo

In [189]:
# Separate features and labels for the full dataset
X = dataframe.iloc[:, :-5].values  # all columns except the last five
y = dataframe['label'].values

# Convert 'cat_id' column to numpy array to be used as groups array for GroupKFold
groups = dataframe['cat_id'].values

# Scale the features using StandardScaler
scaler_full = StandardScaler().fit(X)
X_scaled = scaler_full.transform(X)

# Encode the labels using one-hot encoding
y_encoded = to_categorical(y, num_classes=3)

# Select specific cat_id values for demonstration samples
kitten_cat_id = "050A"
adult_cat_id = "099A"
senior_cat_id = "117A"

# Select all rows corresponding to the sampled cat_id values
demo_samples = dataframe[(dataframe['cat_id'] == kitten_cat_id) | 
                         (dataframe['cat_id'] == senior_cat_id) | 
                         (dataframe['cat_id'] == adult_cat_id)].index

# Convert dataframe indices to positional indices
demo_sample_positions = dataframe.index.get_indexer(demo_samples)

# Separate demonstration samples using positional indices
X_demo = X_scaled[demo_sample_positions]
y_demo = y_encoded[demo_sample_positions]

# Remove demonstration samples from the training set
X_train_full = np.delete(X_scaled, demo_sample_positions, axis=0)
y_train_full = np.delete(y_encoded, demo_sample_positions, axis=0)

# Print label encoding for verification
print("Label encoding:", dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_))))

Label encoding: {'adult': 0, 'kitten': 1, 'senior': 2}


### train

In [190]:
# EarlyStopping callback: monitor 'loss' instead of 'val_loss' for the test set
early_stopping = EarlyStopping(
    monitor='loss',  
    min_delta=0.001, 
    patience=30,  
    verbose=1,  
    restore_best_weights=True  
)

In [191]:
# Define optimizers
optimizers = {
    'Adamax': Adamax(learning_rate=0.003109800273709165)
}

# Compute class weights for the training set
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(np.argmax(y_train_full, axis=1)),
    y=np.argmax(y_train_full, axis=1)
)
weight_dict = {i: class_weights[i] for i in range(len(class_weights))}

# Full model definition with dynamic number of layers
model_full = Sequential()
model_full.add(Dense(128, activation='relu', input_shape=(X_train_full.shape[1],))) 
model_full.add(BatchNormalization())
model_full.add(Dropout(0.44571035356880917))  
model_full.add(Dense(3, activation='softmax'))  

optimizer = optimizers['Adamax']  # optimizer_key from parameters

# Compile the model
model_full.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model on the full training set
history_full = model_full.fit(X_train_full, y_train_full, epochs=1500, batch_size=128,
                              verbose=1, callbacks=[early_stopping], class_weight=weight_dict)

Epoch 1/1500
8/8 [==============================] - 0s 2ms/step - loss: 1.4367 - accuracy: 0.4225
Epoch 2/1500
8/8 [==============================] - 0s 1ms/step - loss: 1.0535 - accuracy: 0.5317
Epoch 3/1500
8/8 [==============================] - 0s 2ms/step - loss: 0.9751 - accuracy: 0.5644
Epoch 4/1500
8/8 [==============================] - 0s 2ms/step - loss: 0.9075 - accuracy: 0.5852
Epoch 5/1500
8/8 [==============================] - 0s 1ms/step - loss: 0.8655 - accuracy: 0.6048
Epoch 6/1500
8/8 [==============================] - 0s 1ms/step - loss: 0.8726 - accuracy: 0.6037
Epoch 7/1500
8/8 [==============================] - 0s 1ms/step - loss: 0.7910 - accuracy: 0.6496
Epoch 8/1500
8/8 [==============================] - 0s 1ms/step - loss: 0.8096 - accuracy: 0.6354
Epoch 9/1500
8/8 [==============================] - 0s 1ms/step - loss: 0.7894 - accuracy: 0.6627
Epoch 10/1500
8/8 [==============================] - 0s 1ms/step - loss: 0.7954 - accuracy: 0.6397
Epoch 11/1500
8/8 [

In [192]:
print(f"Class Weights: {weight_dict}")

Class Weights: {0: 0.6740250183958794, 1: 1.8617886178861789, 2: 1.0211817168338908}


In [193]:
# Save the label mapping
label_mapping = {index: label for index, label in enumerate(label_encoder.classes_)}
print(label_mapping)  # This will print the mapping of labels to encoded values

{0: 'adult', 1: 'kitten', 2: 'senior'}


In [194]:
# Evaluate the model on the training set to get total accuracy
loss, accuracy = model_full.evaluate(X_train_full, y_train_full, verbose=0)
print(f"Total Training Set Accuracy: {accuracy * 100:.2f}%")

# Evaluate the model on the demo set to get accuracy
loss, accuracy = model_full.evaluate(X_demo, y_demo, verbose=0)
print(f"Demo Set Accuracy: {accuracy * 100:.2f}%")

# Predict probabilities for the demo samples
probabilities = model_full.predict(X_demo)

# Convert probabilities to class predictions
predictions = np.argmax(probabilities, axis=1)

# Define the label mapping if not already defined
label_mapping = {0: 'Adult', 1: 'Kitten', 2: 'Senior'}

# Map predictions and actual labels to "Kitten", "Adult", or "Senior" classes
mapped_predictions = [label_mapping[pred] for pred in predictions]
mapped_actual_labels = [label_mapping[np.argmax(label)] for label in y_demo]

# Print out the probabilities along with actual labels and predictions
for i in range(len(probabilities)):
    prob_str = ', '.join([f'{label_mapping[j]}: {prob:.4f}' for j, prob in enumerate(probabilities[i])])
    print(f"Sample {i}: Predicted={mapped_predictions[i]}, Actual={mapped_actual_labels[i]}, Probabilities=({prob_str})")


Total Training Set Accuracy: 85.92%
Demo Set Accuracy: 90.48%
1/1 [==============================] - 0s 25ms/step
Sample 0: Predicted=Adult, Actual=Adult, Probabilities=(Adult: 0.9128, Kitten: 0.0177, Senior: 0.0694)
Sample 1: Predicted=Senior, Actual=Senior, Probabilities=(Adult: 0.1858, Kitten: 0.0019, Senior: 0.8124)
Sample 2: Predicted=Kitten, Actual=Kitten, Probabilities=(Adult: 0.0174, Kitten: 0.9454, Senior: 0.0372)
Sample 3: Predicted=Senior, Actual=Kitten, Probabilities=(Adult: 0.2094, Kitten: 0.3267, Senior: 0.4639)
Sample 4: Predicted=Kitten, Actual=Kitten, Probabilities=(Adult: 0.1410, Kitten: 0.8475, Senior: 0.0115)
Sample 5: Predicted=Senior, Actual=Senior, Probabilities=(Adult: 0.0546, Kitten: 0.0000, Senior: 0.9454)
Sample 6: Predicted=Kitten, Actual=Kitten, Probabilities=(Adult: 0.0426, Kitten: 0.9329, Senior: 0.0245)
Sample 7: Predicted=Senior, Actual=Senior, Probabilities=(Adult: 0.1198, Kitten: 0.0002, Senior: 0.8800)
Sample 8: Predicted=Senior, Actual=Adult, Probab

In [195]:
# Compute the confusion matrix
conf_matrix = confusion_matrix([np.argmax(label) for label in y_demo], predictions)

# Calculate the accuracy per class
class_accuracies = conf_matrix.diagonal() / conf_matrix.sum(axis=1)

# Map the accuracies to class labels
class_accuracy_map = {label_mapping[i]: class_accuracies[i] for i in range(len(class_accuracies))}

# Print the accuracy per class
for class_label, accuracy in class_accuracy_map.items():
    print(f"Accuracy for class {class_label}: {accuracy * 100:.2f}%")

Accuracy for class Adult: 85.71%
Accuracy for class Kitten: 85.71%
Accuracy for class Senior: 100.00%


In [196]:
# Evaluate the model on the training set to get total accuracy
loss, accuracy = model_full.evaluate(X_train_full, y_train_full, verbose=0)
print(f"Total Training Set Accuracy: {accuracy * 100:.2f}%")

# Evaluate the model on the demo set to get accuracy
loss, accuracy = model_full.evaluate(X_demo, y_demo, verbose=0)
print(f"Demo Set Accuracy: {accuracy * 100:.2f}%")

# Predict probabilities for the demo samples
probabilities = model_full.predict(X_demo)

# Convert probabilities to class predictions
predictions = np.argmax(probabilities, axis=1)

# Map predictions and actual labels to "Kitten", "Adult", or "Senior" classes
mapped_predictions = [label_mapping[pred] for pred in predictions]
mapped_actual_labels = [label_mapping[np.argmax(label)] for label in y_demo]

# Print out the probabilities along with actual labels and predictions
for i in range(len(probabilities)):
    class_probabilities = ", ".join([f"{label_mapping[j]}: {prob:.4f}" for j, prob in enumerate(probabilities[i])])
    print(f"Sample {i}: Predicted={mapped_predictions[i]}, Actual={mapped_actual_labels[i]}, Probabilities=({class_probabilities})")


Total Training Set Accuracy: 85.92%
Demo Set Accuracy: 90.48%
1/1 [==============================] - 0s 8ms/step
Sample 0: Predicted=Adult, Actual=Adult, Probabilities=(Adult: 0.9128, Kitten: 0.0177, Senior: 0.0694)
Sample 1: Predicted=Senior, Actual=Senior, Probabilities=(Adult: 0.1858, Kitten: 0.0019, Senior: 0.8124)
Sample 2: Predicted=Kitten, Actual=Kitten, Probabilities=(Adult: 0.0174, Kitten: 0.9454, Senior: 0.0372)
Sample 3: Predicted=Senior, Actual=Kitten, Probabilities=(Adult: 0.2094, Kitten: 0.3267, Senior: 0.4639)
Sample 4: Predicted=Kitten, Actual=Kitten, Probabilities=(Adult: 0.1410, Kitten: 0.8475, Senior: 0.0115)
Sample 5: Predicted=Senior, Actual=Senior, Probabilities=(Adult: 0.0546, Kitten: 0.0000, Senior: 0.9454)
Sample 6: Predicted=Kitten, Actual=Kitten, Probabilities=(Adult: 0.0426, Kitten: 0.9329, Senior: 0.0245)
Sample 7: Predicted=Senior, Actual=Senior, Probabilities=(Adult: 0.1198, Kitten: 0.0002, Senior: 0.8800)
Sample 8: Predicted=Senior, Actual=Adult, Probabi

### Save model

In [197]:
# Save the StandardScaler
joblib.dump(scaler_full, 'scaler_full.pkl')

# Save the trained model
model_full.save('cat_age_model.keras')